In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 61.6 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-10-09 12:41:55--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings
warnings.filterwarnings('ignore')

import re
import seaborn as sb

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score, confusion_matrix

import os

In [3]:
import re 
pattern = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]')
unlabel_data1 = open('/content/drive/MyDrive/Colab Notebooks/data/unlabeled_comments_1.txt', 'r', encoding='utf-8')
unlabel_data2 = open('/content/drive/MyDrive/Colab Notebooks/data/unlabeled_comments_2.txt', 'r', encoding='utf-8')
unlabel_data3 = open('/content/drive/MyDrive/Colab Notebooks/data/unlabeled_comments_3.txt', 'r', encoding='utf-8')
unlabel_data4 = open('/content/drive/MyDrive/Colab Notebooks/data/unlabeled_comments_4.txt', 'r', encoding='utf-8')
unlabel_data5 = open('/content/drive/MyDrive/Colab Notebooks/data/unlabeled_comments_5.txt', 'r', encoding='utf-8')
text=[]

for d in unlabel_data1:
    d = d.strip()
    d = re.sub(pattern, ' ', d)
    text.append(d)
for d in unlabel_data2:
    d = d.strip()
    d = re.sub(pattern, ' ', d)
    text.append(d)
for d in unlabel_data3:
    d = d.strip()
    d = re.sub(pattern, ' ', d)
    text.append(d)
for d in unlabel_data4:
    d = d.strip()
    d = re.sub(pattern, ' ', d)
    text.append(d)
for d in unlabel_data5:
    d = d.strip()
    d = re.sub(pattern, ' ', d)
    text.append(d)

unlabel = pd.DataFrame({'comments': text})

In [4]:
mecab = Mecab()
stopwords = ['에게', '를', '의', '로', '으로', '고', '하고', '듯', '인', '하는', '와', '과', '을', '들', '임', '게', '면', '가', '이가',
             '도', '마저', '한', '면', '네', '이', '는', '하', '다', '은', '지', '에', '거', '나', '보', '는데', '아', '나', '만', '에서',
             '음', '니', '냐', '만', '야', '요', '에요', '든', '걸', '이다', '건', '여']

In [5]:
tokenized_data = []
for sentence in unlabel['comments']:
    temp_X = mecab.morphs(sentence)
    temp_X = [word for word in temp_X if not word in stopwords]
    tokenized_data.append(temp_X)

In [6]:
bigrams = Phrases(sentences=tokenized_data)
trigrams = Phrases(sentences=bigrams[tokenized_data])

In [8]:
embedding_vector_size = 256
trigrams_model = Word2Vec(
    sentences = trigrams[bigrams[tokenized_data]],
    size = embedding_vector_size,
    min_count=3, window=5, workers=4)

In [9]:
print("Vocabulary size:", len(trigrams_model.wv.vocab))

Vocabulary size: 185664


In [17]:
trigrams_model.wv.most_similar('존나')

[('ㅈ_ㄴ', 0.9180233478546143),
 ('졸라', 0.8507997393608093),
 ('겁나', 0.7467561960220337),
 ('존나게', 0.7403463125228882),
 ('조온', 0.6820138692855835),
 ('조_낸', 0.6651002764701843),
 ('줜나', 0.6555241346359253),
 ('졸', 0.6229989528656006),
 ('존', 0.6185598373413086),
 ('겁나게', 0.5937551259994507)]

In [29]:
trigrams_model.save('/content/drive/MyDrive/Colab Notebooks/model/word2vec_trigram_250k.model')

In [20]:
from gensim.models import FastText
trigram_ft_model = FastText(sentences=trigrams[bigrams[tokenized_data]], size=embedding_vector_size,
                            window=5, min_count=3, workers=4, sg=1, min_n=3, max_n=6)

In [28]:
trigram_ft_model.wv.most_similar('개빡치네')

[('려고_녹_취록_까라', 0.8609977960586548),
 ('부절', 0.8582804203033447),
 ('면우', 0.8560243248939514),
 ('ㅜ못', 0.8463903069496155),
 ('혐나', 0.845589816570282),
 ('ㅕㄴ도', 0.8449028730392456),
 ('봤_비겁_살', 0.8442510962486267),
 ('어퍼컷', 0.8430905342102051),
 ('들입다', 0.8427450656890869),
 ('손헌수', 0.8423264622688293)]

In [30]:
trigram_ft_model.save('/content/drive/MyDrive/Colab Notebooks/model/fasttext_trigram_250k.model')

In [1]:
from gensim.models import FastText
ft_model = FastText.load('/content/drive/MyDrive/Colab Notebooks/model/fasttext_trigram_250k.model')

In [6]:
ft_model.most_similar('ㅈㄴ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ㅉㅈㄴ', 0.7807657122612),
 ('저렇게_대놓고', 0.7711836099624634),
 ('흉지', 0.7063531875610352),
 ('썻지', 0.7038073539733887),
 ('잔부', 0.6977627277374268),
 ('애면', 0.6964330673217773),
 ('계정_에다', 0.6959906816482544),
 ('엽사', 0.6935099363327026),
 ('왈_가부', 0.6913948059082031),
 ('가슴_드러내_놓', 0.6898801922798157)]